# Lenet architecture
    
## INPUT => CONV => RELU(TANH) => POOL => CONV => RELU(TANH) => POOL => FC => RELU(TANH) => FC


### import and load data

In [ ]:
# import the necessary packages
from model.cnn.networks.lenet import LeNet
from sklearn.cross_validation import train_test_split
from sklearn import datasets
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np
import argparse
import cv2
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot

# hyper parameters about the model parameters
save_model = 1
load_model = -1
weights_path = "output/lenet_weights.hdf5"

# grab the MNIST dataset
print "[INFO] loading MNIST..."
dataset = datasets.fetch_mldata("MNIST Original")
print "[INFO] finished loading MNIST from ~/scikit_learn_data/mldata/."

data = dataset.data.reshape((dataset.data.shape[0], 28, 28))
data = data[:, np.newaxis, :, :]
(trainData, testData, trainLabels, testLabels) = train_test_split(
    data / 255.0, dataset.target.astype("int"), test_size=0.1)

trainLabels = np_utils.to_categorical(trainLabels, 10)
testLabels = np_utils.to_categorical(testLabels, 10)

### compile the model

In [ ]:
print "[INFO] compiling model..."
opt = SGD(lr=0.01)
model = LeNet.build(width=28, height=28, depth=1, classes=10)

# visualize the network structure
# SVG(model_to_dot(model).create(prog='dot', format='svg'))

model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])
print "[INFO] finished model compiling."

In [ ]:
if load_model < 0:
    print("[INFO] training...")
    model.fit(trainData, trainLabels, batch_size=128, nb_epoch=20,
        verbose=1)
 
    # show the accuracy on the testing set
    print("[INFO] evaluating...")
    (loss, accuracy) = model.evaluate(testData, testLabels,
        batch_size=128, verbose=1)
    print("[INFO] accuracy: {:.2f}%".format(accuracy * 100))
    
if save_model > 0:
    print("[INFO] dumping weights to file...")
    model.save_weights(weights_path, overwrite=True)

In [ ]:
# randomly select a few testing digits
for i in np.random.choice(np.arange(0, len(testLabels)), size=(10,)):
    # classify the digit
    probs = model.predict(testData[np.newaxis, i])
    prediction = probs.argmax(axis=1)
    
    # resize the image from a 28 x 28 image to a 96 x 96 image so we
    # can better see it
    image = (testData[i][0] * 255).astype("uint8")
    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    cv2.putText(image, str(prediction[0]), (5, 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
 
    # show the image and prediction
    print("[INFO] Predicted: {}, Actual: {}".format(prediction[0],
        np.argmax(testLabels[i])))
    cv2.imshow("Digit", image)
    cv2.waitKey(0)